In [86]:
import pandas as pd

In [87]:
df= pd.read_csv("./data/financial_sentiment_analysis.csv")

In [88]:
df

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral
...,...,...
5837,RISING costs have forced packaging producer Hu...,negative
5838,Nordic Walking was first used as a summer trai...,neutral
5839,"According shipping company Viking Line , the E...",neutral
5840,"In the building and home improvement trade , s...",neutral


In [89]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

sw = set(stopwords.words('english'))

def black_txt(token):
    return  token not in sw and token not in list(string.punctuation) and len(token)>2 

def clean_txt(text):
    clean_text = []
    clean_text2 = []
    text = re.sub("'",'',text) 
    text = re.sub("(\\d|\\W)+"," ", text) 
    clean_text = [nltk.stem.WordNetLemmatizer().lemmatize(word,pos="v") for word in word_tokenize(text.lower()) if black_txt(word)]
    clean_text2 = [word for word in clean_text if black_txt(word)] 
    return " ".join(clean_text2) 

In [90]:
dfnew=df[df['Sentiment']!='neutral']

In [91]:
dfnew

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
5,$SPY wouldn't be surprised to see a green close,positive
6,Shell's $70 Billion BG Deal Meets Shareholder ...,negative
...,...,...
5832,Operating profit fell to EUR 38.1 mn from EUR ...,negative
5835,HSBC Says Unit to Book $585 Million Charge on ...,negative
5836,Daily Mail parent company in talks with potent...,positive
5837,RISING costs have forced packaging producer Hu...,negative


In [92]:
dfnew['Sentence']=dfnew['Sentence'].apply(clean_txt)


/var/folders/nw/mj_fmkh561l2yp1b42p5qf480000gn/T/ipykernel_33916/2222403940.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfnew['Sentence']=dfnew['Sentence'].apply(clean_txt)


In [93]:
dfnew

,Sentence,Sentiment
0,geosolutions technology leverage benefon gps s...,positive
1,esi low real possibility,negative
2,last quarter componenta net sales double eur e...,positive
5,spy wouldnt surprise see green close,positive
6,shell billion deal meet shareholder skepticism,negative
...,...,...
5832,operate profit fell eur eur,negative
5835,hsbc say unit book million charge settlement,negative
5836,daily mail parent company talk potential partn...,positive
5837,rise cost force package producer huhtamaki axe...,negative


In [129]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dfnew.Sentence, dfnew.Sentiment, test_size=0.2)

In [131]:
X_train

3613    finnish supplier bss oss vas telecom operators...
3265    adp news feb finnish computer service company ...
5442    jarmo honkamaa head oil refine business neste ...
2080    workforce reduce also amsterdam office netherl...
769     ftse rally three month low boost stanchart sai...
                              ...                        
2047    finnish laboratory liquid handle diagnostic te...
5113    stock little change near record barclays shell...
3375      energy still fail close continue stand weak xle
580     follow level share nflx googl tsla call nice g...
303     autumn plan expand service petersburg moscow l...
Name: Sentence, Length: 2169, dtype: object

In [132]:
X_test

667     announce investment carmaker hyundai example s...
2412    churn high volume ipo price lot sellers try ge...
725     third quarter net sales increase year year eur...
167             tesla recall model cars https bkvstl tsla
3391    moderate significant change result significant...
                              ...                        
5436                             ren bottom good time buy
4019    rdc rowan expand share repurchase program million
1333    olvi control percent stake ragutis coq say ann...
4239               ice say start line finance lse bid war
4555    objective plan measure achieve significant sav...
Name: Sentence, Length: 543, dtype: object

In [163]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

clf = Pipeline([
     ('vectorizer_bow', CountVectorizer(ngram_range = (1, 2))), 
     ('Multi NB', MultinomialNB())
])

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    negative       0.83      0.53      0.65       165
    positive       0.82      0.95      0.88       378

    accuracy                           0.83       543
   macro avg       0.83      0.74      0.77       543
weighted avg       0.83      0.83      0.81       543



In [165]:
dftest = pd.DataFrame()

In [166]:
dftest['Sentence']=X_test

In [167]:
dftest['Sentiment']=y_test

In [168]:
dftest['Prediction']=y_pred

In [169]:
dftest

,Sentence,Sentiment,Prediction
667,announce investment carmaker hyundai example s...,positive,positive
2412,churn high volume ipo price lot sellers try ge...,negative,positive
725,third quarter net sales increase year year eur...,positive,positive
167,tesla recall model cars https bkvstl tsla,negative,negative
3391,moderate significant change result significant...,positive,positive
...,...,...,...
5436,ren bottom good time buy,positive,positive
4019,rdc rowan expand share repurchase program million,positive,positive
1333,olvi control percent stake ragutis coq say ann...,positive,positive
4239,ice say start line finance lse bid war,positive,positive


In [173]:
p=0
for i in y_pred:
    if i=='positive':
        p=p+1
print(p)

print(len(y_pred) - p)


437
106


In [164]:
df['Sentence'][167]

'Tesla is recalling 2,700 Model X cars: https://t.co/8Z7BkVsTl9 $TSLA'